In [ ]:
import os
# Gmail API utils
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
# for encoding/decoding messages in base64
from base64 import urlsafe_b64decode
# for Google Spreadsheets
import gspread_dataframe as gd
import gspread as gs
# Pandas
import pandas as pd
# for pdf parsing
import camelot

# tokens for access to Gmail and Gspreadsheets
TOKEN_GMAIL = 'gmail.json'
TOKEN_GSPREAD = 'gspread.json'
# request all access (permission to read/send/receive emails, manage the inbox, and more)
SCOPES = ['https://mail.google.com/']
# workbook and worksheet
WBOOK = 'bills'
WSHEET = 'bills'

def gmail_authenticate():
    flow = InstalledAppFlow.from_client_secrets_file(TOKEN_GMAIL, SCOPES)
    creds = flow.run_local_server(port=0)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

# get the Google Spreadsheets API service
gc = gs.service_account(filename=TOKEN_GSPREAD)
ws = gc.open(WBOOK).worksheet(WSHEET)

In [5]:
def search_messages(service, query):
    result = service.users().messages().list(userId='me',q=query).execute()
    messages = [ ]
    if 'messages' in result:
        messages.extend(result['messages'])
    while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])
    return messages


def parse_message(service, message):
    '''
    function that downloads attachement of an email and returns date
    '''
    month_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, \
         'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    
    payload = message['payload']

    parts = payload.get('parts')
    if parts:
        for part in parts:
            body = part.get('body')
            part_headers = part.get('headers')

            for part_header in part_headers:
                part_header_name = part_header.get('name')
                part_header_value = part_header.get('value')
                if part_header_name == 'Content-Disposition':
                    if 'attachment' in part_header_value:
                        attachment_id = body.get('attachmentId')
                        attachment = service.users().messages() \
                                        .attachments().get(id=attachment_id, userId='me', messageId=message['id']).execute()
                        data = attachment.get('data')
                        filepath = os.path.join('./tmp/', 'last.pdf')
                        if data:
                            with open(filepath, 'wb') as f:
                                f.write(urlsafe_b64decode(data))
    headers = payload.get('headers')
    if headers:
        for header in headers:
            name = header.get('name')
            value = header.get('value')
            if name == 'Date':
                date = value.split()[2:4]
                month = month_dict.get(date[0])
                year = int(date[1])
                if month != 1:
                    month -= 1
                else:
                    month = 12
                    year -= 1
                return [month, year]


def parse_pdf(period):
    if 'last.pdf' in os.listdir('./tmp/'):
            
        # scan pdf to tables
        tables = camelot.read_pdf('./tmp/last.pdf', strip_text='\n')

        # main data
        df2 = tables[2].df

        # calculate slices
        range1_start = df2.index[df2[0].str.contains('Тех.обсл')][0]
        range1_stop = df2.index[df2[0].str.contains('Домофон')][0] + 1
        range2_start = df2.index[df2[0].str.contains('Холодное')][0]
        range2_stop = df2.index[df2[0].str.contains('Итого к оплате за расчетный период')][0]

        # create df_month
        df_month = pd.concat([df2.iloc[range1_start:range1_stop], df2.iloc[range2_start:range2_stop]])
        df_month.columns = df2.iloc[0]
        df_month.iloc[:,2:15] = df_month.iloc[:,2:15].replace('Х', 0)
        df_month.iloc[:,1:2] = df_month.iloc[:,1:2].replace(['Х', 'кВт'], ['кВт*ч', 'кВт*ч'])
        df_month.iloc[:,:1] = df_month.iloc[:,:1].replace(['Электроэнергия:', '- дн. начисление', \
            '- нч. начисление'], ['Электроэнергия', 'Электроэнергия день', 'Электроэнергия ночь'])
        df_month[['Месяц', 'Год']] = period
        
        return df_month


def mark_as_read(service, query):
    messages_to_mark = query
    print(f'Matched emails: {len(messages_to_mark)}')
    return service.users().messages().batchModify(
      userId='me',
      body={
          'ids': [ msg['id'] for msg in messages_to_mark ],
          'removeLabelIds': ['UNREAD']
      }
    ).execute()

In [ ]:
messages = search_messages(service, 'from:your_email@gmail.com AND subject:Квитанция AND filename:pdf AND is:unread')
if messages:
    mark_as_read(service, messages)

    for message in messages:
        message = service.users().messages().get(userId='me', id=message['id'], format='full').execute()
        payload = message['payload']
        parts = payload.get('parts')
        date = parse_message(service, message)
        existing_data = gd.get_as_dataframe(ws)
        new_data = parse_pdf(date)
        new_data.columns = existing_data.columns
        updated_data = pd.concat([existing_data, new_data])
        gd.set_with_dataframe(ws, updated_data)